In [23]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path

In [24]:
# Path configuration
DATA_DIR = Path("../data/raw/EGATUR")
OUTPUT_DIR = Path("../data/processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [25]:
# Auxiliary functions
def load_elevado_file(filepath: Path) -> pd.DataFrame:
    """Load one 'elevado' file from Egatur"""
    df = pd.read_csv(filepath, sep=";", decimal=".", dtype=str)
    
    for col in ["A13", "gastototal", "factoregatur"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df

def load_etapas_file(filepath: Path) -> pd.DataFrame:
    """Load one 'etapas' file from Egatur"""
    df = pd.read_csv(filepath, sep=";", decimal=".", dtype=str)
    df["pernoc_etapa"] = pd.to_numeric(df["pernoc_etapa"], errors="coerce")
    return df

def process_folder(folder: Path) -> pd.DataFrame:
    """Join elevado + etapas in one specific folder"""
    elevado_file = next(folder.glob("elevado_*.txt"))
    etapas_file = next(folder.glob("etapas_*.txt"))
    
    df_elev = load_elevado_file(elevado_file)
    df_etap = load_etapas_file(etapas_file)
    
    # Aggregate info of etapas by A0_1
    agg_etapas = df_etap.groupby("A0_1").agg(
        etapas_count=("C9_0", "nunique"),
        pernoct_etapas=("pernoc_etapa", "sum")
    ).reset_index()
    
    # Join with elevado dataset
    df = df_elev.merge(agg_etapas, on="A0_1", how="left")
    
    return df

def build_dataset(base_dir: Path) -> pd.DataFrame:
    """Build the complete dataset from all folders"""
    all_dfs = []
    for folder in base_dir.iterdir():
        if folder.is_dir():
            df = process_folder(folder)
            all_dfs.append(df)
    return pd.concat(all_dfs, ignore_index=True)


In [26]:
df = build_dataset(DATA_DIR)

In [27]:
df.describe(include='all')

,mm_aaaa,A0,A0_1,A0_7,A1,pais,ccaa,A13,aloja,motivo,A16,gastototal,factoregatur,etapas_count,pernoct_etapas
count,201360,201360,201360,201360,201360,201360,201360,201360.000000,201360,201360,201360,201360.000000,2.013600e+05,201360.000000,201360.000000
unique,30,1,188770,2,4,14,19,NaN,3,3,2,NaN,NaN,NaN,NaN
top,072023,2,20230500028312,2,2,08,09,NaN,1,1,6,NaN,NaN,NaN,NaN
freq,8946,201360,8,188740,164638,36237,42653,NaN,122987,151511,164720,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.937942,NaN,NaN,NaN,1452.232207,1.109577e+03,1.198451,8.937942
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.557306,NaN,NaN,NaN,1223.239118,1.602653e+03,0.670200,11.557306
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,4.862770,7.406998e-09,1.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,769.712127,2.948826e+02,1.000000,4.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,NaN,NaN,NaN,1167.198694,6.770492e+02,1.000000,6.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,NaN,NaN,1685.772886,1.368035e+03,1.000000,9.000000


## 📑 Dictionary of Original EGATUR Variables

The data comes from the **Tourist Expenditure Survey (EGATUR)** published by the Spanish Statistical Office (INE).  
Each month, two files are provided:

- `elevado_eg_mod_web_tur_*.txt`: information at the **traveler** level (whole trip).
- `etapas_eg_mod_web_*.txt`: information at the **stage** level within a trip.

---

### 🔹 Variables in `elevado_eg_mod_web_tur_*.txt` (trips)

| Variable       | Type     | Values / Example   | Description |
|----------------|----------|-------------------|-------------|
| `mm_aaaa`      | string   | `0123`            | Reference month and year (MMYY). |
| `A0`           | string   | `2`               | Survey source (2 = Egatur). |
| `A0_1`         | string   | `20200200009238`  | **Unique identifier** of the questionnaire (primary key for joins). |
| `A0_7`         | string   | `2`, `8`          | Tourist type: <br>• 2 = Non-resident (non-transit) <br>• 8 = Non-resident in transit. |
| `A1`           | string   | `1-4`             | Exit route: <br>1 = Road, 2 = Airport, 3 = Port, 4 = Train. |
| `pais`         | string   | `01-15`           | Country of residence: <br>01 = Germany, 02 = Belgium, …, 15 = Rest of the world. |
| `ccaa`         | string   | `01-19`           | Main destination Autonomous Community (region): <br>01 = Andalusia, …, 19 = Melilla. |
| `A13`          | integer  | `3`               | Total overnight stays in the trip. |
| `aloja`        | string   | `1-3`             | Main accommodation type: <br>1 = Hotels and similar, 2 = Other market, 3 = Non-market. |
| `motivo`       | string   | `1-3`             | Main trip purpose: <br>1 = Leisure/holidays, 2 = Business, 3 = Other. |
| `A16`          | string   | `1`, `6`          | Package tour: 1 = Yes, 6 = No. |
| `gastototal`   | decimal  | `2341.84`         | Total expenditure of the trip/excursion. |
| `factoregatur` | decimal  | `1998.09`         | Expansion factor (sampling weight). |

**Methodological notes:**
- **Estimated tourist expenditure** = `gastototal * factoregatur`.
- **Number of tourists** = sum of `factoregatur`.
- **Estimated overnight stays** = `A13 * factoregatur`.
- **Daily average expenditure** = total expenditure / estimated overnight stays.
- **Average trip length** = estimated overnight stays / tourists.

---

### 🔹 Variables in `etapas_eg_mod_web_*.txt` (stages)

| Variable       | Type     | Values / Example   | Description |
|----------------|----------|-------------------|-------------|
| `mm_aaaa`      | string   | `0123`            | Reference month and year (MMYY). |
| `A0_1`         | string   | `20200200009238`  | Unique identifier of the questionnaire (**join key with trips**). |
| `C9_0`         | string   | `01`, `02`, …     | Stage identifier within the trip. |
| `ccaa_etapa`   | string   | `01-19`           | Destination Autonomous Community (region) of the stage (same catalog as `ccaa`). |
| `pernoc_etapa` | integer  | `22`              | Number of overnight stays in the stage. |

**Notes:**
- A single `A0_1` may contain multiple stages (e.g., a trip across several regions).
- The join **trips ↔ stages** is done using `A0_1`.

---

### 📌 Source

Spanish Statistical Office (INE):  
[Tourist Expenditure Survey (EGATUR)](https://www.ine.es)  

⚠️ *INE is not responsible for results derived from independent calculations using these files.*  


In [28]:
df.head()

,mm_aaaa,A0,A0_1,A0_7,A1,pais,ccaa,A13,aloja,motivo,A16,gastototal,factoregatur,etapas_count,pernoct_etapas
0,122023,2,20221000016530,2,2,12,10,14,3,1,6,1810.465932,375.936311,1,14
1,122023,2,20221000020837,2,2,12,10,16,3,1,6,2035.596336,324.888905,2,16
2,122023,2,20221000022213,2,2,12,13,9,1,1,6,2460.725851,115.019771,1,9
3,122023,2,20221000039018,2,2,12,10,27,3,1,6,3010.324516,86.451770,3,27
4,122023,2,20221000039098,2,2,12,05,14,3,3,6,1747.853281,236.780128,1,14


## Feature Engineering

In [29]:
# Decode categorical variables
# Decode 'A0_7' (Tourist type)
df["tourist_type"] = df["A0_7"].map({"2": "Non-resident", "8": "Non-resident in transit"})

# Decode 'A1' (Exit route)
df["exit_route"] = df["A1"].map({"1": "Road", "2": "Airport", "3": "Port", "4": "Train"})

# Decode 'pais' (Country of residence)
pais_map = {
    "01": "Germany", "02": "Belgium", "03": "France", "04": "Italy", "05": "Netherlands",
    "06": "Portugal", "07": "United Kingdom", "08": "Switzerland", "09": "Nordic countries",
    "10": "Rest of Europe", "11": "United States", "12": "Rest of America", "13": "Rest of the world",
    "14": "Russia", "15": "Rest of the world"
}
df["pais"] = df["pais"].map(pais_map).fillna(df["pais"])

# Decode 'ccaa' (Main destination Autonomous Community)
ccaa_map = {
    "01": "Andalusia", "02": "Aragon", "03": "Asturias", "04": "Balearic Islands", "05": "Canary Islands",
    "06": "Cantabria", "07": "Castile and Leon", "08": "Castile-La Mancha", "09": "Catalonia",
    "10": "Valencian Community", "11": "Extremadura", "12": "Galicia", "13": "Madrid",
    "14": "Murcia", "15": "Navarre", "16": "Basque Country", "17": "La Rioja", "18": "Ceuta", "19": "Melilla"
}
df["ccaa"] = df["ccaa"].map(ccaa_map).fillna(df["ccaa"])

# Decode 'aloja' (Main accommodation type)
df["aloja"] = df["aloja"].map({"1": "Hotels and similar", "2": "Other market", "3": "Non-market"})

# Decode 'motivo' (Main trip purpose)
df["motivo"] = df["motivo"].map({"1": "Leisure/holidays", "2": "Business", "3": "Other"})

# Decode 'A16' (Package tour)
df["package_tour"] = df["A16"].map({"1": "Yes", "6": "No"})

In [30]:
# Date
df["mm_aaaa"] = df["mm_aaaa"].astype(str)
df["year"] = df["mm_aaaa"].str[-4:].astype(int)
df["month"] = df["mm_aaaa"].str[:-4].astype(int)

# Season
def month_to_season(m):
    if m in [12, 1, 2]:
        return "winter"
    elif m in [3, 4, 5]:
        return "spring"
    elif m in [6, 7, 8]:
        return "summer"
    else:
        return "autumn"

df["season"] = df["month"].apply(month_to_season)

# Derived metrics
df["gasto_ponderado"] = df["gastototal"] * df["factoregatur"]
df["gasto_medio_diario"] = df["gasto_ponderado"] / (df["A13"] * df["factoregatur"])
df["duracion_media"] = (df["A13"] * df["factoregatur"]) / df["factoregatur"]

In [31]:
df.drop(columns=["mm_aaaa", "A0", "A0_1", "A0_7", "A1", "A16", "A13", "etapas_count", "pernoct_etapas"], inplace=True)

In [32]:
df.head()

,pais,ccaa,aloja,motivo,gastototal,factoregatur,tourist_type,exit_route,package_tour,year,month,season,gasto_ponderado,gasto_medio_diario,duracion_media
0,Rest of America,Valencian Community,Non-market,Leisure/holidays,1810.465932,375.936311,Non-resident,Airport,No,2023,12,winter,680619.883798,129.318995,14.0
1,Rest of America,Valencian Community,Non-market,Leisure/holidays,2035.596336,324.888905,Non-resident,Airport,No,2023,12,winter,661342.665150,127.224771,16.0
2,Rest of America,Madrid,Hotels and similar,Leisure/holidays,2460.725851,115.019771,Non-resident,Airport,No,2023,12,winter,283032.122790,273.413983,9.0
3,Rest of America,Valencian Community,Non-market,Leisure/holidays,3010.324516,86.451770,Non-resident,Airport,No,2023,12,winter,260247.882432,111.493501,27.0
4,Rest of America,Canary Islands,Non-market,Other,1747.853281,236.780128,Non-resident,Airport,No,2023,12,winter,413856.923030,124.846663,14.0


In [33]:
df.describe(include='all')

,pais,ccaa,aloja,motivo,gastototal,factoregatur,tourist_type,exit_route,package_tour,year,month,season,gasto_ponderado,gasto_medio_diario,duracion_media
count,201360,201360,201360,201360,201360.000000,2.013600e+05,201360,201360,201360,201360.000000,201360.000000,201360,2.013600e+05,201360.000000,201360.000000
unique,13,19,3,3,NaN,NaN,2,4,2,NaN,NaN,4,NaN,NaN,NaN
top,Switzerland,Catalonia,Hotels and similar,Leisure/holidays,NaN,NaN,Non-resident,Airport,No,NaN,NaN,spring,NaN,NaN,NaN
freq,36237,42653,122987,151511,NaN,NaN,188740,164638,164720,NaN,NaN,57512,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,1452.232207,1.109577e+03,NaN,NaN,NaN,2023.769031,6.132817,NaN,1.462824e+06,237.497803,8.937942
std,NaN,NaN,NaN,NaN,1223.239118,1.602653e+03,NaN,NaN,NaN,0.742960,3.247707,NaN,2.569345e+06,204.398329,11.557306
min,NaN,NaN,NaN,NaN,4.862770,7.406998e-09,NaN,NaN,NaN,2023.000000,1.000000,NaN,2.560074e-05,3.496201,1.000000
25%,NaN,NaN,NaN,NaN,769.712127,2.948826e+02,NaN,NaN,NaN,2023.000000,3.000000,NaN,2.922209e+05,134.392702,4.000000
50%,NaN,NaN,NaN,NaN,1167.198694,6.770492e+02,NaN,NaN,NaN,2024.000000,6.000000,NaN,7.828710e+05,194.566782,6.000000
75%,NaN,NaN,NaN,NaN,1685.772886,1.368035e+03,NaN,NaN,NaN,2024.000000,9.000000,NaN,1.739232e+06,270.108361,9.000000


In [34]:
df.to_csv(OUTPUT_DIR / "egatur_full_dataset.csv", index=False)